In [ ]:
import csv

base_gtfs = "/Users/home/Desktop/mn-hackathon/mtamnrhackathon/MNR_GTFS_Schedules"


In [ ]:
with open(os.path.join(base_gtfs, "agency.txt"), "r", newline="") as infile:
    reader = csv.reader(infile)
    agencies_by_id = {row[0]: row[1] for row in list(reader)[1:]}
agencies_by_id


In [ ]:

with open(os.path.join(base_gtfs, "calendar_dates.txt"), "r", newline="") as infile:
    reader = csv.DictReader(infile)
    service_dates_by_id = {}
    for row in reader:
        if row["exception_type"] == "1":
            if row["service_id"] not in service_dates_by_id:
                service_dates_by_id[row["service_id"]] = []
            service_dates_by_id[row["service_id"]].append(row["date"])
for service_id in list(service_dates_by_id):
    service_dates_by_id[service_id] = sorted(service_dates_by_id[service_id])

service_dates_by_id

In [ ]:
with open(os.path.join(base_gtfs, "notes.txt"), "r", newline="") as infile:
    reader = csv.DictReader(infile)
    notes_by_note_id = {row["note_id"]: row["note_desc"] for row in reader}
notes_by_note_id

In [ ]:
GTFS_ROUTE_TYPES = {
    "0": "Tram",
    "1": "Metro",
    "2": "Rail",
    "3": "Bus",
    "4": "Ferry",
    "5": "Cable tram",
    "6": "Aerial lift",
    "7": "Funicular",
    "11": "Trolleybus",
    "12": "Monorail",
}
with open(os.path.join(base_gtfs, "routes.txt"), "r", newline="") as infile:
    reader = csv.DictReader(infile)
    routes_by_route_id = {row["route_id"]: {
        "route": row["route_long_name"],
        "agency": agencies_by_id[row["agency_id"]],
        "route_type": GTFS_ROUTE_TYPES[row["route_type"]],
        "route_color": row["route_color"],
        "route_text_color": row["route_text_color"],
    }
        for row in reader}
routes_by_route_id

In [13]:
# route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,peak_offpeak


with open(os.path.join(base_gtfs, "trips.txt"), "r", newline="") as infile:
    reader = csv.DictReader(infile)
    trips = []
    for row in reader:
        trip = row.copy()
        trip.update(routes_by_route_id[trip["route_id"]])
        trip.pop("block_id")
        trips.append(trip)

trips



  'trip_short_name': 'B6834',
  'direction_id': '0',
  'shape_id': '34',
  'wheelchair_accessible': '1',
  'peak_offpeak': '0',
  'route': 'Danbury',
  'agency': 'Metro-North Railroad',
  'route_type': 'Rail',
  'route_color': 'EE0034',
  'route_text_color': 'FFFFFF'},
 {'route_id': '5',
  'service_id': '8446',
  'trip_id': '10009011',
  'trip_headsign': 'South Norwalk',
  'trip_short_name': 'B6837',
  'direction_id': '1',
  'shape_id': '33',
  'wheelchair_accessible': '1',
  'peak_offpeak': '0',
  'route': 'Danbury',
  'agency': 'Metro-North Railroad',
  'route_type': 'Rail',
  'route_color': 'EE0034',
  'route_text_color': 'FFFFFF'},
 {'route_id': '5',
  'service_id': '8446',
  'trip_id': '10009012',
  'trip_headsign': 'Danbury',
  'trip_short_name': 'B6846',
  'direction_id': '0',
  'shape_id': '34',
  'wheelchair_accessible': '1',
  'peak_offpeak': '0',
  'route': 'Danbury',
  'agency': 'Metro-North Railroad',
  'route_type': 'Rail',
  'route_color': 'EE0034',
  'route_text_color'

In [ ]:
# stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,wheelchair_boarding
LOCATION_TYPES = {
    "0": "station",
    "1": "entrance/exit",
    "2": "optional/generic",
    "3": "boarding area",
}
with open(os.path.join(base_gtfs, "stops.txt"), "r", newline="") as infile:
    reader = csv.DictReader(infile)
    stops_by_id = {}
    for row in reader:
        stops_by_id[row["stop_id"]] = {
            "stop_code": row["stop_code"],
            "stop_name": row["stop_name"],
            "stop_lat": row["stop_lat"],
            "stop_lon": row["stop_lon"],
            "location_type": LOCATION_TYPES[row["location_type"]],
            "wheelchair_boarding": row["wheelchair_boarding"],
        }
stops_by_id

In [ ]:
# trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,track,note_id

stop_times_by_trip_id = {}
PICKUP_TYPES = {
    "0": True,
    None: True,
    "": True,
    "1": False,
}
DROPOFF_TYPES = {
    "0": True,
    None: True,
    "": True,
    "1": False,
}
with open(os.path.join(base_gtfs, "stop_times.txt"), "r", newline="") as infile:
    reader = csv.reader(infile)
    rows = list(reader)[1:]
    rows.sort(key=lambda row: (row[0], row[4], row[1], row[2]))
    for trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,track,note_id in rows:
        if trip_id not in stop_times_by_trip_id:
            stop_times_by_trip_id[trip_id] = []
        stop_time = {
            "stop_id": stop_id,
            "arrival_time": arrival_time,
            "departure_time": departure_time,
            "stop_sequence": stop_sequence,
            "track": track,
            "drop_off": DROPOFF_TYPES[drop_off_type],
            "pickup": PICKUP_TYPES[pickup_type],
        }
        stop_time.update(stops_by_id[stop_id])
        stop_time["note"] = notes_by_note_id[note_id]
        stop_times_by_trip_id[trip_id].append(stop_time)
stop_times_by_trip_id

In [10]:

fieldnames = set(trips[0].keys())
fieldnames |= set(stop_times_by_trip_id[trips[0]["trip_id"]][0].keys())
fieldnames.add("date")
fieldnames


{'agency',
 'arrival_time',
 'block_id',
 'date',
 'departure_time',
 'direction_id',
 'drop_off',
 'location_type',
 'note',
 'peak_offpeak',
 'pickup',
 'route',
 'route_color',
 'route_id',
 'route_text_color',
 'route_type',
 'service_id',
 'shape_id',
 'stop_code',
 'stop_id',
 'stop_lat',
 'stop_lon',
 'stop_name',
 'stop_sequence',
 'track',
 'trip_headsign',
 'trip_id',
 'trip_short_name',
 'wheelchair_accessible',
 'wheelchair_boarding'}

In [11]:
fieldnames = [
 'route_id',
 'trip_id',
 'service_id',
 'route',
 'date',
 'agency',
 'arrival_time',
 'departure_time',
#  'block_id',
 'direction_id',
 'location_type',
 'route_type',
 'route_color',
 'route_text_color',
 'shape_id',
 'stop_code',
 'stop_id',
 'stop_lat',
 'stop_lon',
 'stop_name',
 'stop_sequence',
 'track',
 'trip_headsign',
 'trip_short_name',
 'pickup',
 'drop_off',
 'peak_offpeak',
 'note',
 'wheelchair_accessible',
 'wheelchair_boarding'
]

In [18]:
# x = trips[0].copy()
# print(x)
import datetime

GTFS_FORMAT = "%Y%m%d"

def floor24(t):
    h, m, s = [int(i) for i in t.split(":")]
    return "{:02}:{:02}:{:02}".format(h % 24, m, s)

with open("timetable.csv", "w", newline="") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    for trip in trips:
        for date in service_dates_by_id[trip["service_id"]]:
            for stop_time in stop_times_by_trip_id[trip["trip_id"]]:
                row = trip.copy()
                row.update(stop_time)
                target_date = date
                time = row["arrival_time"]
                if time >= "24:00:00":
                    dt = datetime.datetime.strptime(date, GTFS_FORMAT)
                    dt += datetime.timedelta(days=1)
                    target_date = dt.strftime(GTFS_FORMAT)
                    row["arrival_time"] = floor24(row["arrival_time"])
                    row["departure_time"] = floor24(row["departure_time"])
                row["date"] = target_date
                writer.writerow(row)

